In [2]:
from pymoo.problems import get_problem
from tqdm import tqdm
import pandas as pd
from utils import *
from pymoo.vendor.vendor_coco import COCOProblem
import random
import os
import seaborn as sns
from pymoo.operators.sampling.lhs import LHS
import polars as pl
import cocoex
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.soo.nonconvex.es import ES

In [16]:
ScaledCOCOProblem(ptype='f-24-15', n_var=5)

In [2]:
def generate_problems():
    suite_year_option = "instances: 1-999"
    #suite_year_option = "instances: 1-500"
    suite_filter_options = (""  # without filtering, a suite has instance_indices 1-15
                             "dimensions: 5"  # skip dimension 40

                           )
    suite = cocoex.Suite('bbob', suite_year_option, suite_filter_options)
    return suite

In [3]:
suite = generate_problems()

In [4]:
#suite[0]

In [5]:
import numpy as np
from pymoo.core.problem import Problem


def algorithm_factory(name, population=20):
    switch = {
        'GA': GA(pop_size=population, sampling=LHS()),
        'PSO': PSO(pop_size=population, sampling=LHS()),
        'DE': DE(pop_size=population, sampling=LHS(), CR=0.1),
        'CMAES': CMAES(popsize=population),
        'ES': ES(pop_size=population),
    }
    algorithm = switch.get(name)
    if algorithm is None:
        raise Exception("Unsupported algorithm: {}".format(name))
    return algorithm

def run_algorithms(problem, n_runs=10, n_eval=2000, algorithms=['GA', 'PSO', 'DE', 'CMAES', 'ES']):
    algo_data = {x: [] for x in algorithms}
    algo_data['algorithm_run'] = []
    for run in range(n_runs):
        algo_data['algorithm_run'].append(run)
        for algo_name in algorithms:
            algorithm = algorithm_factory(algo_name)
            termination = get_termination("n_eval", problem.n_var*n_eval)
            res = minimize(problem, algorithm, termination=termination, verbose=False, save_history=False)
            algo_data[algo_name].append(res.F[0])
    return algo_data



class COCOProblemWrapper(Problem):
    def __init__(self, problem):
        self.coco_problem = problem 
        super().__init__(n_var=self.coco_problem.dimension, n_obj=1, xl=problem.lower_bounds, xu=problem.upper_bounds)

    def _evaluate(self, x, out, *args, **kwargs):
        l = []
        for r in x:
            l.append(self.coco_problem(r))
        l = np.array(l)
        out["F"] = l

#pw = COCOProblemWrapper(suite[0])

In [6]:
#pdf = pl.DataFrame(run_algorithms(pw, n_runs=10, n_eval=1000))

In [7]:
#pdf

In [ ]:
for problem in tqdm(suite):
    file_path = f'new_runs/p_{problem.id_function}__i_{problem.id_instance}.parquet'

    if os.path.exists(file_path):
        continue

    pw = COCOProblemWrapper(problem)
    pdf = pl.DataFrame(run_algorithms(pw, n_runs=10, n_eval=1000)).with_columns([
        pl.lit(problem.id_function).alias("problem"),
        pl.lit(problem.id_instance).alias("instance")
    ])
    
    pdf.write_parquet(file_path)

 15%|█▍        | 3548/23976 [1:04:02<150:09:59, 26.46s/it]

In [ ]:
#pdf

In [ ]:




if os.path.isfile(file):
    df = pl.read_parquet(file)
else:

    from scipy.stats import qmc
    from tqdm import tqdm

    data = {'problem': [], 'instance': [], 'y':[]}
    for problem in tqdm(suite):
        sampling = LHS()
        sampler = qmc.LatinHypercube(d=problem.dimension)
        sample = sampler.random(n=250*problem.dimension)
        sample_scaled = qmc.scale(sample, problem.lower_bounds, problem.upper_bounds)
        data['problem'].extend(len(sample_scaled)*[problem.id_function])
        data['instance'].extend(len(sample_scaled)*[problem.id_instance])
        l = []
        for r in range(problem.dimension):
            if f'x_{r}' not in data:
                data[f'x_{r}'] = []
            data[f'x_{r}'].extend(sample_scaled[:, r])
        for s in sample_scaled:
            v = problem(s)
            l.append(v)
        data['y'].extend(l)
    df = pl.DataFrame(data)
    df.write_parquet('samples.parquet')

In [ ]:
df.filter(pl.col('problem')==1).filter(pl.col('instance')==1)['y'].min()

In [ ]:
#def basic_features(df):
#    # Group by 'problem' and 'instance' to calculate statistics for each group
#    grouped_df = df.group_by(['problem', 'instance']).agg([
#        (pl.col('y').max() - pl.col('y').min()).alias('y_min_max_diff'),
#        pl.col('y').quantile(0.10).alias('y_10_quantile'),
#        pl.col('y').quantile(0.25).alias('y_25_quantile'),
#        pl.col('y').quantile(0.50).alias('y_50_quantile'),
#        pl.col('y').quantile(0.75).alias('y_75_quantile'),
#        pl.col('y').quantile(0.90).alias('y_90_quantile')
#    ]).sort(by=['problem', 'instance'])
#    
#    return grouped_df

In [ ]:
import polars as pl

def basic_features(df):
    # First, normalize the 'y' values within each group (problem, instance)
    df = df.with_columns([
        (pl.col('y') - pl.col('y').min()).over(['problem', 'instance']).alias('y_min_max_norm')
    ]).with_columns([
        (pl.col('y_min_max_norm') / pl.col('y_min_max_norm').max()).over(['problem', 'instance']).alias('y_normalized')
    ])
    
    # Calculate mean of y_normalized for later use
    df = df.with_columns([
        pl.col('y_normalized').mean().over(['problem', 'instance']).alias('y_mean')
    ])
    
    # Apply sin and cos to y_normalized and scaled versions
    df = df.with_columns([
        pl.col('y_normalized').sin().alias('y_sin'),
        pl.col('y_normalized').cos().alias('y_cos'),
        (pl.col('y_normalized') * 5).sin().alias('y_sin_5'),
        (pl.col('y_normalized') * 5).cos().alias('y_cos_5'),
        (pl.col('y_normalized') * 10).sin().alias('y_sin_10'),
        (pl.col('y_normalized') * 10).cos().alias('y_cos_10'),
        (pl.col('y_normalized') * 50).sin().alias('y_sin_50'),
        (pl.col('y_normalized') * 50).cos().alias('y_cos_50')
    ]).with_columns([
        pl.col('y_sin').mean().over(['problem', 'instance']).alias('y_sin_mean'),
        pl.col('y_cos').mean().over(['problem', 'instance']).alias('y_cos_mean'),
        pl.col('y_sin_5').mean().over(['problem', 'instance']).alias('y_sin_5_mean'),
        pl.col('y_cos_5').mean().over(['problem', 'instance']).alias('y_cos_5_mean'),
        pl.col('y_sin_10').mean().over(['problem', 'instance']).alias('y_sin_10_mean'),
        pl.col('y_cos_10').mean().over(['problem', 'instance']).alias('y_cos_10_mean'),
        pl.col('y_sin_50').mean().over(['problem', 'instance']).alias('y_sin_50_mean'),
        pl.col('y_cos_50').mean().over(['problem', 'instance']).alias('y_cos_50_mean')
    ])
    
    # Apply various power transformations
    df = df.with_columns([
        (pl.col('y_normalized') ** 0.5).alias('y_power_1_2'),
        (pl.col('y_normalized') ** (1/3)).alias('y_power_1_3'),
        (pl.col('y_normalized') ** (1/4)).alias('y_power_1_4'),
        (pl.col('y_normalized') ** (1/5)).alias('y_power_1_5'),
        (pl.col('y_normalized') ** 2).alias('y_power_2'),
        (pl.col('y_normalized') ** 3).alias('y_power_3'),
        (pl.col('y_normalized') ** 4).alias('y_power_4'),
        (pl.col('y_normalized') ** 5).alias('y_power_5')
    ])
    
    # Group by 'problem' and 'instance' and compute features
    grouped_df = df.group_by(['problem', 'instance']).agg([
        (pl.col('y_normalized').max() - pl.col('y_normalized').min()).alias('y_min_max_diff'),
        pl.col('y_normalized').quantile(0.10).alias('y_10_quantile'),
        pl.col('y_normalized').quantile(0.25).alias('y_25_quantile'),
        pl.col('y_normalized').quantile(0.50).alias('y_50_quantile'),
        pl.col('y_normalized').quantile(0.75).alias('y_75_quantile'),
        pl.col('y_normalized').quantile(0.90).alias('y_90_quantile'),
        
        # Count values greater than the mean
        (pl.col('y_normalized') > pl.col('y_mean')).mean().alias('y_larger_than_mean'),
        
        # Sum of squared values
        (pl.col('y_normalized') ** 2).sum().alias('mean_of_squares'),
        
        # Mean values of sin, cos, and the power-transformed features
        pl.col('y_sin_mean').mean().alias('mean_y_sin'),
        pl.col('y_cos_mean').mean().alias('mean_y_cos'),
        pl.col('y_sin_5_mean').mean().alias('mean_y_sin_5'),
        pl.col('y_cos_5_mean').mean().alias('mean_y_cos_5'),
        pl.col('y_sin_10_mean').mean().alias('mean_y_sin_10'),
        pl.col('y_cos_10_mean').mean().alias('mean_y_cos_10'),
        pl.col('y_sin_50_mean').mean().alias('mean_y_sin_50'),
        pl.col('y_cos_50_mean').mean().alias('mean_y_cos_50'),
        pl.col('y_power_1_2').mean().alias('mean_y_power_1_2'),
        pl.col('y_power_1_3').mean().alias('mean_y_power_1_3'),
        pl.col('y_power_1_4').mean().alias('mean_y_power_1_4'),
        pl.col('y_power_1_5').mean().alias('mean_y_power_1_5'),
        pl.col('y_power_2').mean().alias('mean_y_power_2'),
        pl.col('y_power_3').mean().alias('mean_y_power_3'),
        pl.col('y_power_4').mean().alias('mean_y_power_4'),
        pl.col('y_power_5').mean().alias('mean_y_power_5')
    ])
    
    return grouped_df


In [ ]:
fdf = df.pipe(basic_features)
fdf

In [32]:
.3**(1/5)

0.7860030855966228

In [ ]:
import seaborn as sns

In [ ]:
sns.boxenplot(data=fdf, x="problem", y="y_min_max_diff")
plt.yscale('log')

In [ ]:
selected_columns = [x for x in list(fdf.columns) if x.startswith('mean')]
selected_columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

train_df, test_df = train_test_split(fdf.to_pandas(), test_size=0.3, random_state=42)

# Step 2: Separate features and target
X_train = train_df[selected_columns]
y_train = train_df['problem']
X_test = test_df[selected_columns]
y_test = test_df['problem']

# Step 3: Train a RandomForest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Step 4: Predict on the test set
y_pred = clf.predict(X_test)

# Step 5: Evaluate the model
print(classification_report(y_test, y_pred))

In [ ]:
fdf = fdf.sort(['problem','instance'])
fdf

In [ ]:
optimization_columns = ["GA", "PSO", "DE", "CMAES", "ES"]
pdf = pl.read_parquet('new_runs/*').sort(['problem','instance'])
pdf = pdf.with_columns([
    pl.col("problem").cast(pl.Int64),
    pl.col("instance").cast(pl.Int64)
])
pdf

In [ ]:
def get_ranks(pdf):
    n = pdf.select(optimization_columns).to_pandas().rank(axis=1)
    n['algorithm_run'] = pdf['algorithm_run']
    n['problem'] = pdf['problem']
    n['instance'] = pdf['instance']
    return pl.DataFrame(n)#.sort(['problem','instance'])

def get_per_instance_ranks(pdf):
    rdf = get_ranks(pdf)
    return rdf.group_by(['problem', 'instance']).mean().sort(['problem','instance'])

rdf = get_per_instance_ranks(pdf)
rdf

In [ ]:
joined = rdf.join(fdf, on=["problem", "instance"], how="inner")
joined

In [ ]:
train_df, test_df = joined.filter(pl.col('instance')!=1), joined.filter(pl.col('instance')==1)

X_train = train_df[selected_columns]
y_train = train_df[optimization_columns]
X_test = test_df[selected_columns]
y_test = test_df[optimization_columns]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)

In [ ]:
mean_squared_error(y_test.to_numpy(), model.predict(X_test))

In [ ]:
from sklearn.dummy import DummyRegressor
model = DummyRegressor()
model.fit(X_train, y_train)

In [ ]:
mean_squared_error(y_test.to_numpy(), model.predict(X_test))

In [ ]:
#y_train